In [ ]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
from tensorflow.keras import losses
from tensorflow.keras import layers
import numpy as np

Performed Sentimental analysis on movie reviews using
Artificial Neural Network with an accuracy of 87%


In [ ]:
url='https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
dataset=tf.keras.utils.get_file('aclImdb_v1',url,untar=True,cache_dir='.',cache_subdir='')
dataset_dir=os.path.join(os.path.dirname(dataset),'aclImdb')

84125825/84125825 [==============================] - 6s 0us/step


In [ ]:
os.listdir(dataset_dir)
#here we do not need to divide data into train and test.The data is already divided into train and test in different folders

['imdb.vocab', 'test', 'README', 'imdbEr.txt', 'train']

In [ ]:
dataset_dir

'./aclImdb'

In [ ]:
train_dir=os.path.join(dataset_dir,'train')
os.listdir(train_dir)

['urls_neg.txt',
 'urls_pos.txt',
 'labeledBow.feat',
 'urls_unsup.txt',
 'pos',
 'unsup',
 'unsupBow.feat',
 'neg']

In [ ]:
sample_file=os.path.join(train_dir,'pos/0_9.txt')
with open(sample_file) as f:
  print(f.read())

Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High's satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I'm here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn't!


In [ ]:
#removing directories that are not required
remove_dir=os.path.join(train_dir,'unsup')
shutil.rmtree(remove_dir)

In [ ]:
os.listdir(train_dir)

['urls_neg.txt',
 'urls_pos.txt',
 'labeledBow.feat',
 'urls_unsup.txt',
 'pos',
 'unsupBow.feat',
 'neg']

In [ ]:
batch_size=32#32 samples/rows/text files per iteration
seed=42#it is like random_state field in model building in ML

In [ ]:
#dataset object.you cannot query it as it is not an enumerable.it also contains label associated with data like they are positive or negative reviews
raw_train_ds=tf.keras.utils.text_dataset_from_directory('aclImdb/train',
    batch_size=batch_size,
                                                        validation_split=0.2,
                                                        subset='training',
                                                        seed=seed
)
#2 classes are positive reviews and negative reviews
#5000 files for validation dataset and 20,000 files for testing dataset
'''validation dataset is required because suppose you build your model and train it for 1000 epochs(without validation dataset) and now test the model with testing dataset and conclude
your model is not good so now you will have to change the model and train it again for 1000 epochs.This is is a very tiring and long process,Rather after end of each epoch if you test
your model with the validation dataset and print both training and validation losses everytime and understand if the model is overfitting so you can encounter the problem first and wont
have to go again with the tedious process.'''

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


'validation dataset is required because suppose you build your model and train it for 1000 epochs(without validation dataset) and now test the model with testing dataset and conclude\nyour model is not good so now you will have to change the model and train it again for 1000 epochs.This is is a very tiring and long process,Rather after end of each epoch if you test\nyour model with the validation dataset and print both training and validation losses everytime and understand if the model is overfitting so you can encounter the problem first and wont\nhave to go again with the tedious process.'

In [ ]:
for text_batch,label_batch in raw_train_ds.take(1):#take one patch/sample/text file at random from training dataset
  for i in range(3):
    print('Review ',text_batch.numpy()[i])
    print('Label ',label_batch.numpy()[i])

Review  b'"Pandemonium" is a horror movie spoof that comes off more stupid than funny. Believe me when I tell you, I love comedies. Especially comedy spoofs. "Airplane", "The Naked Gun" trilogy, "Blazing Saddles", "High Anxiety", and "Spaceballs" are some of my favorite comedies that spoof a particular genre. "Pandemonium" is not up there with those films. Most of the scenes in this movie had me sitting there in stunned silence because the movie wasn\'t all that funny. There are a few laughs in the film, but when you watch a comedy, you expect to laugh a lot more than a few times and that\'s all this film has going for it. Geez, "Scream" had more laughs than this film and that was more of a horror film. How bizarre is that?<br /><br />*1/2 (out of four)'
Label  0
Review  b"David Mamet is a very interesting and a very un-equal director. His first movie 'House of Games' was the one I liked best, and it set a series of films with characters whose perspective of life changes as they get in

In [ ]:
print('Label 0 corresponds to: ',raw_train_ds.class_names[0])
print('Label 1 corresponds to: ',raw_train_ds.class_names[1])

Label 0 corresponds to:  neg
Label 1 corresponds to:  pos


In [ ]:
raw_val_ds=tf.keras.utils.text_dataset_from_directory('aclImdb/train',
    batch_size=batch_size,
                                                        validation_split=0.2,
                                                        subset='validation',
                                                        seed=seed
)

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [ ]:
raw_test_ds=tf.keras.utils.text_dataset_from_directory('aclImdb/test',
    batch_size=batch_size,

)

Found 25000 files belonging to 2 classes.


In [ ]:
sample_file=os.path.join(train_dir,'pos/3_10.txt')
t=open(sample_file)
a=t.read()
a

'"All the world\'s a stage and its people actors in it"--or something like that. Who the hell said that theatre stopped at the orchestra pit--or even at the theatre door? Why is not the audience participants in the theatrical experience, including the story itself?<br /><br />This film was a grand experiment that said: "Hey! the story is you and it needs more than your attention, it needs your active participation". "Sometimes we bring the story to you, sometimes you have to go to the story."<br /><br />Alas no one listened, but that does not mean it should not have been said.'

In [ ]:
lowercase=tf.strings.lower(a)
lowercase

<tf.Tensor: shape=(), dtype=string, numpy=b'"all the world\'s a stage and its people actors in it"--or something like that. who the hell said that theatre stopped at the orchestra pit--or even at the theatre door? why is not the audience participants in the theatrical experience, including the story itself?<br /><br />this film was a grand experiment that said: "hey! the story is you and it needs more than your attention, it needs your active participation". "sometimes we bring the story to you, sometimes you have to go to the story."<br /><br />alas no one listened, but that does not mean it should not have been said.'>

In [ ]:
stripped_html=tf.strings.regex_replace(lowercase,'<br />','')
stripped_html

<tf.Tensor: shape=(), dtype=string, numpy=b'"all the world\'s a stage and its people actors in it"--or something like that. who the hell said that theatre stopped at the orchestra pit--or even at the theatre door? why is not the audience participants in the theatrical experience, including the story itself?this film was a grand experiment that said: "hey! the story is you and it needs more than your attention, it needs your active participation". "sometimes we bring the story to you, sometimes you have to go to the story."alas no one listened, but that does not mean it should not have been said.'>

In [ ]:
def custom_standardization(input_data):
  lowercase=tf.strings.lower(a)
  stripped_html=tf.strings.regex_replace(lowercase,'<br />','')
  return tf.strings.regex_replace(stripped_html,'[%s]' % re.escape(string.punctuation),'')#remove all punctuations

In [ ]:
#now to convert words from the reviews to numbers to be used in the mathematical equation of our model
#There are 2 ways to do this:
#1st way:
#Using word vectorizer:We combine all the words from all the reviews/rows/data points/text files and make a list of all the unique words.
#You will make a table where the list of unique words would be the columns of the table and each review would be the rows.Now for every word you will count the occurence of that
#word in the review.For the words that are not present,their number will be zero.Now these numbers for each unique word will be your input to the neural network.
#You will repeat this process for each review.
#This way is not used for our neural network in tensorflow

In [ ]:
#This next way is used in our neural network
#Word embeddings/Word indexing:We combine all the words from all the reviews/rows/data points/text files and make a list of all the unique words.
#You assign any random number ,having any range, to the unique words each.They can be decimals or integers.

In [ ]:
max_features=10000
sequence_length=250
vectorize_layer=layers.TextVectorization(standardize=custom_standardization,max_tokens=max_features,output_mode='int',output_sequence_length=sequence_length)

In [ ]:
train_text=raw_train_ds.map(lambda x,y:x)#train_text contains two columns review and labels.labels contains pos and neg corresponding to the review which is positive or negative
vectorize_layer.adapt(train_text)#vectorize the reviews using word embeddings function mentioned in the previous cell

In [ ]:
def vectorize_text(text,label):
  text=tf.expand_dims(text,-1)#reshaping the dataset
  return vectorize_layer(text),label

In [ ]:
text_batch,label_batch=next(iter(raw_train_ds))
first_review,first_label=text_batch[0],label_batch[0]
print('Review: ',first_review)
print('Label: ',raw_train_ds.class_names[first_label])
print('Vectorized review: \n',vectorize_text(first_review,first_label))

Review:  tf.Tensor(b'Great movie - especially the music - Etta James - "At Last". This speaks volumes when you have finally found that special someone.', shape=(), dtype=string)
Label:  neg
Vectorized review: 
 (<tf.Tensor: shape=(250,), dtype=int64, numpy=
array([62,  2, 20, 19, 29, 18, 44, 33, 63, 15, 45, 30, 42,  3, 22,  2, 48,
        7,  3, 11, 28, 17,  2, 36, 32, 54, 17,  2, 11, 55, 21, 14,  8,  2,
       60, 35, 15,  2, 25, 53, 46,  2,  6, 43, 51, 24, 19, 49, 52,  3,  7,
       47,  2,  6, 14,  4, 18,  9, 13, 39, 26, 10, 61,  9, 13, 10, 64, 34,
       12, 23, 58,  2,  6,  5,  4, 12,  4, 16,  5, 50,  5,  2, 27, 38, 37,
       41, 57,  3, 56,  8, 40,  9, 31,  8, 16, 59,  7,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 

In [ ]:
print('1: ',vectorize_layer.get_vocabulary()[1])
# print('67: ',vectorize_layer.get_vocabulary()[67])
# print('400: ',vectorize_layer.get_vocabulary()[400])
print(len(vectorize_layer.get_vocabulary()))

1:  [UNK]
65


In [ ]:
#dataset object
raw_train_ds = tf.keras.utils.text_dataset_from_directory('aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

#the take(1) method takes 1 batch from the dataset
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(3):
        print("Review", text_batch.numpy()[i])
        #When you use .numpy() on a TensorFlow tensor, you are converting the tensor to a NumPy array
        print("Label", label_batch.numpy()[i])

raw_val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)

raw_test_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/test',
    batch_size=batch_size)

def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(stripped_html,'[%s]' % re.escape(string.punctuation),'')

max_features = 10000
sequence_length = 250

vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

# Make a text-only dataset (without labels), then call adapt
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)


# a function to see how the above vectorization is happening
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

print(" 1 ---> ",vectorize_layer.get_vocabulary()[1])
print(" 2 ---> ",vectorize_layer.get_vocabulary()[2])
print(" 3 ---> ",vectorize_layer.get_vocabulary()[3])
print("400 ---> ",vectorize_layer.get_vocabulary()[400])
print("5000 ---> ",vectorize_layer.get_vocabulary()[5000])
print("8000 ---> ",vectorize_layer.get_vocabulary()[8000])
print('Vocabulary size: {}'.format(len(vectorize_layer.get_vocabulary())))



Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Review b'"Pandemonium" is a horror movie spoof that comes off more stupid than funny. Believe me when I tell you, I love comedies. Especially comedy spoofs. "Airplane", "The Naked Gun" trilogy, "Blazing Saddles", "High Anxiety", and "Spaceballs" are some of my favorite comedies that spoof a particular genre. "Pandemonium" is not up there with those films. Most of the scenes in this movie had me sitting there in stunned silence because the movie wasn\'t all that funny. There are a few laughs in the film, but when you watch a comedy, you expect to laugh a lot more than a few times and that\'s all this film has going for it. Geez, "Scream" had more laughs than this film and that was more of a horror film. How bizarre is that?<br /><br />*1/2 (out of four)'
Label 0
Review b"David Mamet is a very interesting and a very un-equal director. His first movie 'House of Games' was the one I liked best, and it set a series of

In [ ]:
train_ds=raw_train_ds.map(vectorize_text)
val_ds=raw_val_ds.map(vectorize_text)
test_ds=raw_test_ds.map(vectorize_text)

In [ ]:
AUTOTUNE=tf.data.AUTOTUNE
train_ds=train_ds.cache().prefetch(buffer_size=AUTOTUNE)
Val_ds=val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds=test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
embedding_dim=16

In [ ]:
model=tf.keras.Sequential([
   layers.Embedding(max_features,embedding_dim),
   layers.Dropout(0.2),
   layers.GlobalAveragePooling1D(),
   layers.Dropout(0.2),
   layers.Dense(1),
])
model.summary()
#so now this neural network structure is different from the previous neural network structure discussed.Here we have input shape of(1,250) where each review will be a vectorized array
#of 250 numbers.Now each value of 250 numbers for one input will have 16(embedding_dim value) weights associated to it so now the input structure is (1,250,16) as each word or
#number out of 250 values has 16 weights associated to it.So the structure is 1 row which contains a cell with contains 250 rows and 16 columns.Now in GloablAveragePooling1D
#Layer we take average of each 16 columns and get the shape as (1,16) Now in the final layer there is just one element which will be sum of all the 16 columns from before.This will
#be probability lying between [0,1] and closer to 0 will be a negative review and vice-versa.embedding_dim is user defined and can contain any value.

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          160000    
                                                                 
 dropout (Dropout)           (None, None, 16)          0         
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 160017 (625.07 KB)
Trainable params: 160017 (625.07 KB)
Non-trainable params: 0 (0.00 Byte)
________________

In [ ]:
model.compile(loss=losses.BinaryCrossentropy(from_logits=True),optimizer='adam',metrics=tf.metrics.BinaryAccuracy(threshold=0.0))
#DOUBT:what use it will be of setting threshold zero,all reviews will be marked positive or class 1

In [ ]:
epochs=10
history=model.fit(train_ds,validation_data=val_ds,epochs=epochs)

Epoch 1/10
625/625 [==============================] - 6s 8ms/step - loss: 0.6650 - binary_accuracy: 0.6880 - val_loss: 0.6162 - val_binary_accuracy: 0.7706
Epoch 2/10
625/625 [==============================] - 3s 5ms/step - loss: 0.5504 - binary_accuracy: 0.8010 - val_loss: 0.4994 - val_binary_accuracy: 0.8220
Epoch 3/10
625/625 [==============================] - 3s 5ms/step - loss: 0.4457 - binary_accuracy: 0.8440 - val_loss: 0.4209 - val_binary_accuracy: 0.8466
Epoch 4/10
625/625 [==============================] - 3s 5ms/step - loss: 0.3795 - binary_accuracy: 0.8652 - val_loss: 0.3745 - val_binary_accuracy: 0.8598
Epoch 5/10
625/625 [==============================] - 3s 5ms/step - loss: 0.3364 - binary_accuracy: 0.8797 - val_loss: 0.3453 - val_binary_accuracy: 0.8668
Epoch 6/10
625/625 [==============================] - 3s 5ms/step - loss: 0.3062 - binary_accuracy: 0.8882 - val_loss: 0.3263 - val_binary_accuracy: 0.8722
Epoch 7/10
625/625 [==============================] - 3s 5ms/ste

In [ ]:
loss,accuracy=model.evaluate(test_ds)
print('Loss: ',loss)
print('Accuracy: ',accuracy)

782/782 [==============================] - 2s 3ms/step - loss: 0.3102 - binary_accuracy: 0.8732
Loss:  0.310169517993927
Accuracy:  0.8732399940490723


In [ ]:
export_model=tf.keras.Sequential([
    vectorize_layer,
    model,
    layers.Activation('sigmoid')
])
export_model.compile(loss=losses.BinaryCrossentropy(from_logits=False),optimizer='adam',metrics=['accuracy'])
loss,accuracy=export_model.evaluate(raw_test_ds)
print('Loss: ',loss)
print('Accuracy: ',accuracy)

782/782 [==============================] - 3s 4ms/step - loss: 0.3102 - accuracy: 0.8732
Loss:  0.31016993522644043
Accuracy:  0.8732399940490723


In [ ]:
examples=['The movie was great!','The movie was okay.','I hated the movie.','I really want a better movie to watch.']
export_model.predict(examples)

1/1 [==============================] - 0s 103ms/step


array([[0.63824546],
       [0.46190187],
       [0.49973133],
       [0.52188975]], dtype=float32)